<a href="https://colab.research.google.com/github/stalgiag/DAIN/blob/master/DAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check for CUDA

In [0]:
!nvcc --version

Install Pytorch

In [0]:
# different torch install needed
# !pip3 uninstall torch
!pip3 install -U https://download.pytorch.org/whl/cu100/torch-1.0.0-cp36-cp36m-linux_x86_64.whl

Check Pytorch Install Works with Cuda

In [0]:
!python -c "import torch; print(torch.__version__); print(torch.cuda.is_available())"

In [0]:
# we will verify that GPU is enabled for this notebook
# following should print: CUDA is available!  Training on GPU ...
# 
# if it prints otherwise, then you need to enable GPU: 
# from Menu > Runtime > Change Runtime Type > Hardware Accelerator > GPU

import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [0]:
# check that gcc is installed
# should print version number along bottom
!gcc -v

Clone my modified fork

In [0]:
#clone repo - my fork has a few simple mods that make this work with colab
!git clone https://github.com/stalgiag/DAIN.git

Setup

In [0]:
#move into repo and setup
#may take awhile
%cd DAIN
%cd my_package
!./build.sh
%cd ../PWCNet/correlation_package_pytorch1_0
!./build.sh

In [0]:
%cd ../../

In [0]:
!mkdir model_weights
!mkdir MiddleBurySet

In [0]:
%cd model_weights
!wget http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

In [0]:
%cd ../MiddleBurySet
!wget http://vision.middlebury.edu/flow/data/comp/zip/other-color-allframes.zip
!unzip other-color-allframes.zip
!wget http://vision.middlebury.edu/flow/data/comp/zip/other-gt-interp.zip
!unzip other-gt-interp.zip
%cd ..

In [0]:
!nvidia-smi

In [0]:
#test that it works
!CUDA_VISIBLE_DEVICES=0 python demo_MiddleBury.py

In [0]:
# slow motion fun
# get rid of the uid flag if you don't want this to overwrite
!CUDA_VISIBLE_DEVICES=0 python demo_MiddleBury_slowmotion.py --netName DAIN_slowmotion --time_step 0.1 --uid 111

In [0]:

%cd MiddleBurySet/other-result-author/111/Beanbags
!apt-get install imagemagick
!convert -delay 1 -loop 0 *.png beanbag.gif

This builds 10x slomo gif at  `DAIN/MiddleBurySet/other-result-author/111/Beanbags/beanbag.gif`